## LLM Integration

Once all the steps were developed:

- Embedding Service
- Ingestion Pipeline
- Context retrieval

Now its time to create the last part of the RAG-LLM technique. Send the context and the user's query to the LLM in order to the LLM to generate an answer.

This time, I'll be using ChatGPT LLM's, but also can work with Google LLM and others.


In [21]:
from google import genai
from google.genai import types

import sys

sys.path.append("..")

In [2]:
from rag_llm_energy_expert.credentials import get_qdrant_config, get_llm_config

In [3]:
from rag_llm_energy_expert.search.searchers import semantic_search

In [4]:
qdrant_config=get_qdrant_config()
llm_config=get_llm_config()
collection_name = qdrant_config.COLLECTION_NAME + qdrant_config.COLLECTION_VERSION

## Connecting the GENAI client

code from: https://ai.google.dev/gemini-api/docs/text-generation

In [7]:
llm_client = genai.Client(api_key=llm_config.API_KEY.get_secret_value())

Generating multi-turn conversations

The chat format enables users to step incrementally toward answers and to get help with multipart problems.

In [15]:
# Creates a new chat session
chat = llm_client.chats.create(model=llm_config.MODEL)

In [18]:
response = chat.send_message("Hi, im 25 years old")
print(response.text)

Okay! That's great. Is there anything specific you'd like to talk about or need help with? Knowing you're 25 doesn't give me much context, but I'm happy to help in any way I can. For example, are you looking for:

*   **Advice on something specific?** (career, relationships, finances, etc.)
*   **Information about something?** (hobbies, travel, current events, etc.)
*   **Ideas for something?** (gifts, activities, etc.)
*   **Just someone to chat with?**

Let me know!



In [19]:
response = chat.send_message("If I am 5 years older than my sister. How old is she?")
response.text

"You can't determine your sister's exact age with just that information. You only know that she is 5 years younger than you.\n\nSince you are 25, your sister is 20 years old.\n"

In [20]:
for message in chat.get_history():
    print(f'role - {message.role}',end=": ")
    print(message.parts[0].text)

role - user: Hi, tell me a joke
role - model: Why don't scientists trust atoms?

Because they make up everything!

role - user: Hi, tell me a joke
role - model: Why did the bicycle fall over? 

Because it was two tired!

role - user: Hi, im 25 years old
role - model: Okay! That's great. Is there anything specific you'd like to talk about or need help with? Knowing you're 25 doesn't give me much context, but I'm happy to help in any way I can. For example, are you looking for:

*   **Advice on something specific?** (career, relationships, finances, etc.)
*   **Information about something?** (hobbies, travel, current events, etc.)
*   **Ideas for something?** (gifts, activities, etc.)
*   **Just someone to chat with?**

Let me know!

role - user: If I am 5 years older than my sister. How old is she?
role - model: You can't determine your sister's exact age with just that information. You only know that she is 5 years younger than you.

Since you are 25, your sister is 20 years old.



### Configuring parameters

Every prompt sent to the model includes parameters that control how the model generates responses. You can configure these parameters, por let the model use the default options

In [22]:
# Creates a new chat session
chat2 = llm_client.chats.create(
    model=llm_config.MODEL,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0.1
    )
    )

In [23]:
responses = chat2.send_message(
    message = "Hi, If I have 5 apples, and 2 pears, and for all of them I paid 10 USD, if the apples costs 1 USD, how much are the pears?"
)
print(responses.text)

Here's how to solve the problem:

*   **Cost of apples:** 5 apples * $1/apple = $5
*   **Cost of pears:** $10 (total) - $5 (apples) = $5
*   **Cost per pear:** $5 / 2 pears = $2.50/pear

**Answer:** The pears cost $2.50 each.


More model parameters can be found [here](https://ai.google.dev/gemini-api/docs/text-generation)

### System Instructions

System instructions let you steer the behaviour of a model baesd on you specific use case. When you provide system instructions, you give the model additional context to help it understand the task and generate more customized responses. The model should adhere to the system instructions over the full iteraction with the user, enabling you to specify product-level behaviour separete from the prompts provided by end users.

In [25]:
# Creates a new chat session
chat3 = llm_client.chats.create(
    model=llm_config.MODEL,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0.1,
        system_instruction="You are a Mexican energy expert that solves doubts of clients. You must be as direct as possible. Your responses" \
        "shall not be longer than 2 paragraphs (5 lines each)." \
        "The responses shall be based on the context provided. If you don't know the answer, tell that you don't know." \
        "Answer the user's questions in the same language as they're asked."
    )
    )

In [41]:
query = "En el nuevo modelo, cómo se considera a Pemex?"

Semantic Search of the available info in the vector DB

In [42]:

print(semantic_search(
    query=query,
    embedding_model_name=None,
    chunk_overlap=0,
    documents_limit=5,
    collection_name=collection_name
))

2025-04-18 19:12:19.900 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:37 - Preprocessing query...
2025-04-18 19:12:19.904 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:74 - Generating embeddings...
2025-04-18 19:12:24.531 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:87 - Embeddings generated successfully
2025-04-18 19:12:24.531 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:95 - Preparing embeddings for vector search
2025-04-18 19:12:24.531 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:106 - Query preprocessed successfully
2025-04-18 19:12:24.830 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:120 - Processing query results...
2025-04-18 19:12:24.835 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:131 - Query results processed



adjudicado el contrato. Cuando una asignación migre a un contrato podrá llevarse a cabo una 
asociación entre Pemex y un partícular, y la CNH realizará una licitación para elegir al socio 
(incluyendo las asignaciones de la Ronda Cero).
• El modelo propuesto también considera que Pemex podrá migrar a la nueva modalidad de 
contratación

 Pemex en la industria petrolera. Mediante 
la “Ronda Cero”, Pemex podrá elegir aquellos campos en producción y aquellas áreas en ex-
ploración que tengan interés en operar y donde demuestre tener capacidad técnica, financie-
ra y de ejecución para desarrollarlos en forma eficiente y competitiva y podrá migrarlas hacia 
un esqu

 pensiones y jubilaciones 
de PEMEX y CFE, sujeto a que acuerden con sus trabajadores un nuevo régimen de pensiones 
que reduzca esos pasivos y la Auditoria Superior de la Federación audite la evolución de di-
chos pasivos.
• Los Consejos de Administración de ambas empresas tendrán una nueva estructura organi-
zacional y se enc

In [40]:
question = "PEMEX podrá migrar a la nueva modalidad de contratación"

response = chat3.send_message(message=question,
                              config = types.GenerateContentConfig(
                                  temperature=0.5,
                                  system_instruction="You are a Mexican energy expert that solves doubts of clients. Your responses" \
        "shall not be longer than 2 paragraphs (5 lines each)." \
        "The responses shall be based on the context provided. If you don't know the answer, tell that you don't know." \
        "Answer the user's questions in the same language as they're asked. Try to generate friendly answers"\
        f"""Context: {semantic_search(query=question,
                                    embedding_model_name=None,
                                    chunk_overlap=0,
                                    collection_name = collection_name,
                                    documents_limit = 5
                                    )}"""
                              ))
print(response.text)

2025-04-18 19:11:21.954 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:37 - Preprocessing query...
2025-04-18 19:11:21.958 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:74 - Generating embeddings...
2025-04-18 19:11:26.236 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:87 - Embeddings generated successfully
2025-04-18 19:11:26.245 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:95 - Preparing embeddings for vector search
2025-04-18 19:11:26.246 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:106 - Query preprocessed successfully
2025-04-18 19:11:26.547 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:120 - Processing query results...
2025-04-18 19:11:26.552 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:131 - Query results processed


¡Hola! Sí, Pemex podrá migrar a la nueva modalidad de contratación. Esto le permitirá asociarse con particulares, incluso en asignaciones de la Ronda Cero, mediante licitaciones supervisadas por la CNH para elegir al socio más adecuado.

